In [1]:
from tqdm import tqdm
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker

import numpy as np
import os, glob
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=1
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle
import argparse, subprocess
from importlib import reload
import cmasher as cmr
from copy import deepcopy

database = f"/home/jeon/MissingSat/database/nh2"

mode = 'nh2'
iout = 797
repo, rurmode, dp = mode2repo(mode)
snap = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snaps = uri.TimeSeries(snap)
snaps.read_iout_avail()
nout = snaps.iout_avail['iout']
gals = uhmi.HaloMaker.load(snap, galaxy=True, double_precision=dp)
hals = uhmi.HaloMaker.load(snap, galaxy=False, double_precision=dp)
pouts = snaps.iout_avail['iout'][snaps.iout_avail['age'] >= snap.age-1]
pouts = pouts[pouts < snap.iout][::-1]
print(pouts)

[Output 00797] Age (Gyr) : 11.623 / 13.741, z = 0.17158 (a = 0.8536)
[796 795 794 793 792 791 790 789 788 787 786 785 784 783 782 781 780 779
 778 777 776 775 774 773 772 771 770 769 768 767 766 765 764 763 762 761
 760 759 758 757 756 755 754 753 752 751 750 749 748 747 746 745 744 743
 742 741 740 739 738 737 736 735 734 733 732 731 730 729 728 727 726 725
 724 723 722 721 720 719 718 717 716 715 714 713 712]


In [2]:
def get_members(gal, galaxy=True):
    global members, snaps
    if(gal['timestep'] in members.keys()):
        if(gal['id'] in members[gal['timestep']].keys()):
            return members[gal['timestep']][gal['id']]
    else:
        members[gal['timestep']] = {}
    members[gal['timestep']][gal['id']] = uhmi.HaloMaker.read_member_part(snaps.get_snap(gal['timestep']), gal['id'], galaxy=galaxy, simple=True)
    return members[gal['timestep']][gal['id']]

In [8]:
LGs = pklload(f"{database}/LG")
for LGkey in LGs.keys():
    subs_lets_check = LGs[LGkey]['subs']['id']
    LG = LGs[LGkey]

    BGG = LG['BGG']


    rrange = BGG['r']
    uri.timer.verbose=0
    centers = {}
    members = {}
    all_scores = {}
    give_scores = {}
    take_scores = {}
    tmp = uhmi.HaloMaker.read_member_parts(snap, hals[subs_lets_check-1], galaxy=False, nthread=16, target_fields=['id'])
    my_members = {}
    for subid in subs_lets_check:
        my_members[subid] = tmp[tmp['hmid']==subid]['id']
    # for ip, pout in tqdm( enumerate(pouts), total=len(pouts) ):
    for ip, pout in enumerate(pouts):
        print(f"[{pout:04d}]")
        psnap = snaps.get_snap(pout)
        pgals = uhmi.HaloMaker.load(psnap, galaxy=False)

        for subid in subs_lets_check:
            sub = hals[subid-1]
            if(sub['id'] in centers.keys()):
                center = centers[sub['id']]
            else:
                center = [sub['x'], sub['y'], sub['z']]
                centers[sub['id']] = center
            my_member = my_members[subid]
            pneighbors = cut_box(pgals, *center, rrange)


            if(len(pneighbors)==0): continue

            give_score = np.zeros(len(pneighbors))
            take_score = np.zeros(len(pneighbors))
            pmembers = uhmi.HaloMaker.read_member_parts(psnap, pneighbors, galaxy=False, nthread=16, target_fields=['id'])
            for i, pg in enumerate(pneighbors):
                pmember = pmembers[pmembers['hmid']==pg['id']]['id']
                intersect = np.sum( isin(pmember, my_member, assume_unique=True) )
                give_score[i] = intersect / len(my_member) / 2
                take_score[i] = intersect / len(pmember) / 2
            all_score = give_score * take_score
            
            argmax_all = np.argmax(all_score)
            argmax_give = np.argmax(give_score)
            argmax_take = np.argmax(take_score)
            if(not sub['id'] in all_scores.keys()):
                all_scores[sub['id']] = np.zeros(len(pouts))
                give_scores[sub['id']] = np.zeros(len(pouts))
                take_scores[sub['id']] = np.zeros(len(pouts))
            
            all_scores[sub['id']][ip] = pneighbors['id'][argmax_all] + all_score[argmax_all]
            give_scores[sub['id']][ip] = pneighbors['id'][argmax_give] + give_score[argmax_give]
            take_scores[sub['id']][ip] = pneighbors['id'][argmax_take] + take_score[argmax_take]
            centers[sub['id']] = [ pneighbors['x'][argmax_all], pneighbors['y'][argmax_all], pneighbors['z'][argmax_all] ]
        psnap.clear(part=True)
    pklsave(give_scores, f"./nh2_give_dm_scores_host{BGG['id']:04d}.pickle", overwrite=True)
    pklsave(take_scores, f"./nh2_take_dm_scores_host{BGG['id']:04d}.pickle", overwrite=True)
    uri.timer.verbose=1

[0796]
<function unique at 0x7fb91820cb80>
[ 6760980  6858714  6871789  6871927  6876368  6876964  6877175  6882188
  6883899  6887037  6887259  6887882  6888464  6888702  6891963  6892464
  6893348  6893578  6903098  6903340  6903584  6903827  6910544  6910740
  6910934  6916704  6919983  6965979  6973565  6977758  6977994  6983429
  6984378  6984621  6988783  6988999  6989623  6989838  6992937  6993165
  6993430  6993678  6994174  6997353  6999893  7001980  7008512  7008744
  7009403  7009634  7010887  7012359  7012603  7013316  7013559  7017208
  7017442  7017669  7017898  7018367  7018594 11891919 11892171 11893153
 11893224 11893458 11893688 11893931 11894170 11894621 11894857 11897285
 11897520 11897765 11898000 11898067 11898245 11898479 11898546 11898725
 11901110 11901295 11901366 11901622 11901652 11901878 11902134 11902319
 11902389 11902645 11902830 11902900 11905106 11905359 11905613 11905866
 11906121 11906373 11906628 11906880 11906945 11907368 11907809 11908232
 1191065

ValueError: stop!